In [1]:
from tqdm import tqdm
import numpy as np
from numpy.linalg import matrix_power
import networkx as nx
import scipy as sp
from copy import deepcopy

## Part 1

In [ ]:
total_map = open("Day21_input_filled.txt").read().splitlines()
n_rows = len(total_map)
n_cols = len(total_map[0])


G = nx.Graph()
nodes_dict = {}

n_nodes_bef_S = 0
# add nodes
for ydx in tqdm(range(n_rows)):
    for xdx in range(n_cols):
        if total_map[ydx][xdx]  in 'S.':
            node_idx = n_cols*ydx + xdx
            G.add_node(node_idx)
            # keep track of which node (number) is S
            if total_map[ydx][xdx] == 'S':
                S_graph_numb = n_nodes_bef_S
            # add edges left and top
            if node_idx -1 in G.nodes:
                G.add_edge(node_idx - 1, node_idx)
            if node_idx -n_cols in G.nodes:
                G.add_edge(node_idx - n_cols, node_idx)
            n_nodes_bef_S +=1
            # keep track of which node number is in which map position
            nodes_dict[(ydx, xdx)] = n_nodes_bef_S
            
# generate adjacency map            
adj_mat = nx.adjacency_matrix(G)


n_power = 1
new_mat = deepcopy(adj_mat)
# 64 is 2^6, so I only need ro multiply new_mat for itself 6 times
for n_mult in tqdm(range(1,7)):
    n_power *=2
    new_mat = new_mat.dot(new_mat)
    print('after steps taken =', n_power, ' possible locations = ', sum(new_mat.toarray()[S_graph_numb] !=0))    

dense_mat = new_mat.toarray()

# numbers get so big that they go in overflow, so >0 does not work
print(sum(dense_mat[S_graph_numb] !=0))

# map generation visualization
temp_total_map = deepcopy(total_map)
for ydx in tqdm(range(n_rows)):
    for xdx in range(n_cols):
        # if I am on a node
        if total_map[ydx][xdx]  in 'S.':
            # compute the number of the node in the adjacency matrix
            n_node = nodes_dict[(ydx, xdx)]
            # if the node is connected to S within X steps
            if dense_mat[S_graph_numb][n_node-1] !=0:
                # substitute the symbol with a 'O'
                temp_line = list(temp_total_map[ydx])
                temp_line[xdx] = 'O'
                temp_total_map[ydx] = "".join(temp_line)        
                    
# for visualization
with open('Day21_graph_out.txt', 'w') as f:
    for line in temp_total_map:
        f.write(line +'\n')

## Part 2

In [116]:
def compute_steps(S_pos, n_steps, matrix):
    possible_steps_after_n_steps = 0
    n_rows = len(matrix)
    n_cols = len(matrix[0])
    for ydx in range(n_rows):
        for xdx in range(n_cols):
             if matrix[ydx][xdx] in 'S.':
                    tiles_dist = abs(ydx - S_pos[0]) + abs(xdx - S_pos[1])
                    if tiles_dist <= n_steps and not(tiles_dist-n_steps)%2:
                    #if tiles_dist > n_steps and not(tiles_dist-n_steps)%2:
                        possible_steps_after_n_steps +=1
                        #temp_line = list(updated_map[ydx])
                        #temp_line[xdx] = 'O'
                        #updated_map[ydx] = "".join(temp_line)
    
    return possible_steps_after_n_steps

In [99]:
# Day21_input has been modified by removing the ~6 '.' that were not connected to S
# Under the assumption that everything is connected, the algorithm is much faster

total_map = open("Day21_input_filled.txt").read().splitlines()
n_rows = len(total_map)
n_cols = len(total_map[0])

updated_map = deepcopy(total_map)
max_n_steps = 26501365


S_pos = [65,65]
#S_pos = [n_rows -1 , n_cols -1]
n_steps = 65
possible_steps_after_n_steps = 0
for ydx in tqdm(range(n_rows)):
    for xdx in range(n_cols):
         if total_map[ydx][xdx] in 'S.':
                tiles_dist = abs(ydx - S_pos[0]) + abs(xdx - S_pos[1])
                
                if tiles_dist <= n_steps and not(tiles_dist-n_steps)%2:
                #if tiles_dist > n_steps and not(tiles_dist-n_steps)%2:
                    possible_steps_after_n_steps +=1
                    temp_line = list(updated_map[ydx])
                    temp_line[xdx] = 'O'
                    updated_map[ydx] = "".join(temp_line)


print(possible_steps_after_n_steps)

# for visualization
with open('Day21_map.txt', 'w') as f:
    for line in updated_map:
        f.write(line +'\n')

# n_steps even -> 7481
# n_steps odd -> 7407
top_left = 911 # n_steps = 63
top_right = 917 # n_steps = 63
bot_left = 921 # n_steps = 63
bot_right = 906 # n_steps = 63
n_inside_65 = 3752

n_outside_65_diam = top_left + top_right + bot_left  + bot_right
print(n_outside_65_diam)
print(n_outside_65_diam + n_inside_65)

100%|██████████████████████████████████████████████████████████████████████████████| 131/131 [00:00<00:00, 8381.83it/s]

3752
3655
7407


In [100]:
big_step = 26501365

n_steps_in_last_gardens = (big_step-S_pos[0])%n_cols
n_full_garden_crossed = int((big_step-n_steps_in_last_gardens)/n_rows)
print('in last gardens we do steps ', n_steps_in_last_gardens)
print('full gardens crossed NSWE = ', n_full_garden_crossed)

tiles_in_full_garden_odd = 7407
tiles_in_65_steps = 3752
tiles_after_65_steps = 3655
tiles_in_full_garden_minus_65_steps = tiles_in_full_garden_odd - tiles_in_65_steps
print('tiles after 65 steps ', tiles_after_65_steps)
print('tiles besides 65 steps ', tiles_in_full_garden_minus_65_steps)

in last gardens we do steps  0
full gardens crossed NSWE =  202300
tiles after 65 steps  3655
tiles besides 65 steps  3655


In [132]:

# 1 + 4*n*(n-1)/2
n_full_squares =  1 + 4*n_full_garden_crossed*(n_full_garden_crossed+1)/2 # --> times 3752
# 4*n*(n-1)/2
n_empty_squares = + 4*n_full_garden_crossed*(n_full_garden_crossed+1)/2 # --> times 3655

total_steps = n_full_squares*tiles_in_65_steps + n_empty_squares*tiles_in_full_garden_minus_65_steps
print(int(total_steps))

# 606270242935855 is too low
# 606270242935952 not correct
# 609298746763952 is correct

diff = 609298746763952 - 606270242935952
error_per_square = diff/202300**2

print(error_per_square)
print('close enough')

606270242935952
74.00079090459714
close enough


In [117]:
nine_matrices =[]
for ydx in range(n_cols):
    nine_matrices.append(total_map[ydx]*3)
nine_matrices = nine_matrices*3


S_pos_nine_mat = [131+65, 131+65]
n_steps_nine_mat = 131+65

coverage_nine_mat = compute_steps(S_pos_nine_mat, n_steps_nine_mat, nine_matrices)
print(coverage_nine_mat)

33614


In [118]:
twenty5_matrices =[]
for ydx in range(n_cols):
    twenty5_matrices.append(total_map[ydx]*5)
twenty5_matrices = twenty5_matrices*5


S_pos_25_mat = [131*2+65, 131*2+65]
n_steps_25_mat = 131*2+65

coverage_25_mat = compute_steps(S_pos_25_mat, n_steps_25_mat, twenty5_matrices)
print(coverage_25_mat)

93252


With the above numbers, I could generate the string {{0 , 3752}, {1 , 33614},{ 2 , 93252 }} to put in wolframAlpha and generate an interpolate square function, which I then computed in x = 202300